This file contains the Extract, Transform, and Load (ETL) process to convert cleaned data into a structured data warehouse schema. The goal is to prepare the data for business analysis, visualization, and predictive modeling.

Dimension Modeling

In [ ]:
import pandas as pd
import numpy as np

# --- Assumption: df_rawatinap and df_rawatjalan are already loaded from the previous file. ---

# 1. Patient Dimension (dim_pasien)
# Combines and standardizes patient data from both inpatient and outpatient datasets.
# rm_id serves as the unique business key.
dim_pasien_inap = df_rawatinap[['rm_id', 'jenis_kelamin', 'umur', 'kelompok_umur', 'tipe_pasien']].drop_duplicates().copy()
dim_pasien_jalan = df_rawatjalan[['rm_id', 'jenis_kelamin', 'tipe_pasien']].copy()
dim_pasien_jalan['umur'] = np.nan
dim_pasien_jalan['kelompok_umur'] = np.nan
dim_pasien_jalan = dim_pasien_jalan[['rm_id', 'jenis_kelamin', 'umur', 'kelompok_umur', 'tipe_pasien']]

dim_pasien = pd.concat([dim_pasien_inap, dim_pasien_jalan], ignore_index=True)
dim_pasien = dim_pasien.sort_values(by='rm_id').drop_duplicates(subset='rm_id', keep='first').reset_index(drop=True)
dim_pasien['pasien_id'] = dim_pasien.index + 1
dim_pasien = dim_pasien[['pasien_id', 'rm_id', 'jenis_kelamin', 'umur', 'kelompok_umur', 'tipe_pasien']]
print("Patient Dimension created.")
display(dim_pasien.head())

# 2. Payment Method Dimension (dim_cara_bayar)
# Retrieves unique combinations of all payment methods.
dim_cara_bayar = pd.concat([
    df_rawatinap[['cara_bayar', 'cara_bayar_kelompok']],
    df_rawatjalan[['cara_bayar']].rename(columns={'cara_bayar': 'cara_bayar_kelompok'}).copy()
], ignore_index=True).drop_duplicates().dropna().reset_index(drop=True)
dim_cara_bayar.rename(columns={'cara_bayar': 'cara_bayar_nama'}, inplace=True)
dim_cara_bayar['cara_bayar_id'] = dim_cara_bayar.index + 1
dim_cara_bayar = dim_cara_bayar[['cara_bayar_id', 'cara_bayar_nama', 'cara_bayar_kelompok']]
print("Payment Method Dimension created.")
display(dim_cara_bayar.head())

# 3. Procedure Dimension (dim_tindakan)
# Combines all procedures from both inpatient and outpatient datasets.
rawatinap_tindakan = df_rawatinap[['tindakan']].copy()
rawatjalan_tindakan = pd.melt(
    df_rawatjalan[['tindakan', 'tindakan_2', 'tindakan_3', 'tindakan_4', 'tindakan_5', 'tindakan_6']],
    value_name='tindakan'
)
dim_tindakan = pd.concat([rawatinap_tindakan, rawatjalan_tindakan[['tindakan']]], ignore_index=True)
dim_tindakan = dim_tindakan.dropna(subset=['tindakan']).drop_duplicates().reset_index(drop=True)
dim_tindakan['tindakan_id'] = dim_tindakan.index + 1
dim_tindakan = dim_tindakan[['tindakan_id', 'tindakan']]
print("Procedure Dimension created.")
display(dim_tindakan.head())

# 4. Diagnosis Dimension (dim_diagnosis)
# Combines primary and secondary diagnoses from all datasets.
diagnosis_inap = df_rawatinap[['icd_10', 'arti_icd']].rename(columns={'icd_10': 'kode_icd10'}).copy()
diagnosis_jalan_utama = df_rawatjalan[['icd_10']].rename(columns={'icd_10': 'kode_icd10'}).copy()
diagnosis_jalan_tambahan = pd.melt(
    df_rawatjalan[['kode_2', 'kode_3', 'kode_4', 'kode_5', 'kode_6']],
    value_name='kode_icd10'
)
dim_diagnosis = pd.concat([diagnosis_inap[['kode_icd10', 'arti_icd']], diagnosis_jalan_utama, diagnosis_jalan_tambahan], ignore_index=True)
dim_diagnosis = dim_diagnosis.drop_duplicates(subset='kode_icd10').dropna(subset=['kode_icd10']).reset_index(drop=True)
dim_diagnosis['diagnosis_id'] = dim_diagnosis.index + 1
dim_diagnosis = dim_diagnosis[['diagnosis_id', 'kode_icd10', 'arti_icd']]
print("Diagnosis Dimension created.")
display(dim_diagnosis.head())

# 5. Doctor Dimension (dim_dokter)
dim_dokter = pd.concat([df_rawatinap[['dokter']], df_rawatjalan[['dokter']]], ignore_index=True)
dim_dokter = dim_dokter.drop_duplicates().reset_index(drop=True)
dim_dokter['dokter_id'] = dim_dokter.index + 1
dim_dokter = dim_dokter[['dokter_id', 'dokter']]
print("Doctor Dimension created.")
display(dim_dokter.head())

# 6. Disposition Status Dimension (dim_disposisi)
dim_disposisi = pd.concat([df_rawatinap[['status_disposisi']], df_rawatjalan[['status_disposisi']]], ignore_index=True)
dim_disposisi = dim_disposisi.dropna().drop_duplicates().reset_index(drop=True)
dim_disposisi['disposisi_id'] = dim_disposisi.index + 1
dim_disposisi = dim_disposisi[['disposisi_id', 'status_disposisi']]
print("Disposition Status Dimension created.")
display(dim_disposisi.head())

# 7. Poly Dimension (dim_poli)
dim_poli = df_rawatjalan[['nama_poli']].drop_duplicates().reset_index(drop=True)
dim_poli['poli_id'] = dim_poli.index + 1
dim_poli = dim_poli[['poli_id', 'nama_poli']]
print("Poly Dimension created.")
display(dim_poli.head())

# 8. Time Dimension (dim_waktu)
all_tanggal = pd.concat([
    df_rawatinap[['tanggal_admisi']].rename(columns={'tanggal_admisi': 'tanggal'}),
    df_rawatinap[['tanggal_keluar']].rename(columns={'tanggal_keluar': 'tanggal'}),
    df_rawatjalan[['tanggal']]
], ignore_index=True)
all_tanggal = all_tanggal.dropna().drop_duplicates().reset_index(drop=True)
all_tanggal['bulan'] = all_tanggal['tanggal'].dt.month
all_tanggal['tahun'] = all_tanggal['tanggal'].dt.year
all_tanggal['tanggal_id'] = all_tanggal.index + 1
dim_waktu = all_tanggal[['tanggal_id', 'tanggal', 'bulan', 'tahun']]
print("Time Dimension created.")
display(dim_waktu.head())

Fact Table Modeling

In [ ]:
# 1. Inpatient Fact Table (fakta_rawatinap)
# Merges the inpatient data with all dimension tables using foreign keys.
fakta_rawatinap = df_rawatinap.merge(dim_pasien[['rm_id', 'pasien_id']], on='rm_id', how='left')
fakta_rawatinap = fakta_rawatinap.merge(dim_dokter, on='dokter', how='left')
# ... (continue with merges to other dimensions)

# Reorder columns for the final fact table
fakta_rawatinap = fakta_rawatinap[[
    'pasien_id', 'dokter_id', 'cara_bayar_id', 'tanggal_admisi_id', 'tanggal_keluar_id',
    'diagnosis_id', 'tindakan_id', 'disposisi_id', 'jumlah_kunjungan_inap', 'lama_rawat'
]]
fakta_rawatinap['rawat_inap_id'] = fakta_rawatinap.index + 1
print("Inpatient Fact Table created.")

# 2. Outpatient Fact Table (fakta_rawatjalan)
# Merges the outpatient data with all dimension tables using foreign keys.
fakta_rawatjalan = df_rawatjalan.merge(dim_pasien[['rm_id', 'pasien_id']], on='rm_id', how='left')
# ... (continue with merges to other dimensions)

# Reorder columns for the final fact table
fakta_rawatjalan = fakta_rawatjalan[[
    'pasien_id', 'dokter_id', 'cara_bayar_id', 'tanggal_kunjungan_id',
    'diagnosis_id', 'tindakan_id', 'disposisi_id', 'poli_id', 'jumlah_kunjungan_jalan'
]]
fakta_rawatjalan['rawat_jalan_id'] = fakta_rawatjalan.index + 1
print("Outpatient Fact Table created.")